# Useful links
    https://developer.spotify.com/documentation/general/guides/authorization-guide/
    https://www.base64encode.org/
    https://developer.spotify.com/documentation/web-api/reference/search/search/


In [1]:
import requests
import pandas as pd
import base64

In [2]:
client_id = 'Your client id'
client_secret = 'Your client secret'
url_base = "https://api.spotify.com/v1"


In [3]:
token_url = 'https://accounts.spotify.com/api/token'


# Manual Token

In [4]:
# #code64 = your_64coded_info
# params = {'grant_type': 'client_credentials'}
# headers = {'Authorization':f'Basic {code64}'}
# r = requests.post(token_url, data=params, headers=headers)
# print(f'Token status code: {r.status_code}')

# token = r.json()['access_token']
# header = {'Authorization': f'Bearer {token}'}

## Get artist info with a manual id

In [5]:
# #https://open.spotify.com/artist/0zgFL90nGTrH2iOMD8Vysy Sabino
# artist_id = '0zgFL90nGTrH2iOMD8Vysy'
# ep_artist = f'/artists/{artist_id}'

# url_artist = url_base+ ep_artist
# print(f'url: {url_artist}')

# r = requests.get(url_artist, headers = header)
# print(f'Request status code: {r.status_code}')
# r.json()


## Get id by search

In [6]:
def search_artist_id(client_id, client_secret, search_name):
    token = get_token(client_id,client_secret)
    header = {'Authorization': f'Bearer {token}'}
    params = {'grant_type':'client_credentials'}

    search_url = 'https://api.spotify.com/v1/search'
    search_params = {'q': search_name, 'type':'artist', 'market':'MX'}
    search = requests.get(search_url, headers = header, params = search_params)
    print(f'Search status code: {search.status_code}')
    
    df = pd.DataFrame(search.json()['artists']['items'])
    df.sort_values(by='popularity', ascending = False)
    artist_id = df.iloc[0].id
    return artist_id


# Get token function

In [7]:
def get_token(client_id, client_secret):
    encoded = base64.b64encode(bytes(f'{client_id}:{client_secret}','utf-8'))
    params = {'grant_type':'client_credentials'}
    header = {'Authorization': f"Basic {str(encoded,'utf-8')}"}
    r = requests.post(token_url, headers = header, data = params)
    if r.status_code != 200:
        print(f'Request Error: {r.json()} ')
        return None
    else:
        #print(f"Token valid for {r.json()['expires_in']}secs")
        return r.json()['access_token']
    

# Getting discography
## Getting albums

In [8]:
def get_discography(artist_id, client_id,client_secret, page_limit = 50, country = None):
    token = get_token(client_id, client_secret)
    header = {'Authorization': f'Bearer {token}'}
    
    endpt_albums = f'/artists/{artist_id}/albums'
    url_albums = url_base + endpt_albums
    params = {'country': 'MX'}

    try:
        artist_albums = requests.get(url_albums, headers = header, params = params)
        albums_items = artist_albums.json()['items']
    except:
        print(f'Request error: {artist_albums.json()}')
        return None
    while artist_albums.json()['next']:
        artist_albums = requests.get(artist_albums.json()['next'], headers = header)
        albums_items+=artist_albums.json()['items']
    return albums_items
#get_discography(artist_id, client_id,client_secret)

## Get tracks in an album

In [19]:
def get_album_tracks(client_id,client_secret, album_id):
    token = get_token(client_id, client_secret)
    header = {'Authorization': f'Bearer {token}'}
    endpt_album_tracks = f'/albums/{album_id}/tracks'
    url_album_tracks = url_base + endpt_album_tracks
    params = {'market': 'MX'}   
    album_info =  requests.get(url_album_tracks, headers=header, params=params)
    #print(f'Album info status code: {album_info.status_code}')
    album_info = album_info.json()['items']
    #return [{'name': track['name'],'id':track['id']} for track in album_info]
    return {track['name']:{'id':track['id'], 'preview': track['preview_url']} for track in album_info}


# Main

In [20]:
artist_name = 'Sabino'
artist_id = search_artist_id(client_id,client_secret, artist_name)
artist_albums = get_discography(artist_id, client_id,client_secret)
albums_list = {album['name']:{'id':album['id']} for album in artist_albums}

#getting tracks info per album
for album in albums_list:
    albums_list[album]['tracks'] = get_album_tracks(client_id,client_secret, albums_list[album]['id'])
albums_list
#

Search status code: 200


{'Yin': {'id': '53H9AYzn3j647Tr3JnGI71',
  'tracks': {'La Magia': {'id': '7dnLbhOrka4iruYAk7XwJA',
    'preview': 'https://p.scdn.co/mp3-preview/5fbaa8a13c125f2d855b8a482a11c7fc0546a124?cid=c5e5d6eec0ee45518a572285d4a12260'},
   'Conmigo Siempre': {'id': '1omuLiHsrZNngTOHr2FF9U',
    'preview': 'https://p.scdn.co/mp3-preview/b1784ed63c8d81129817d0e882d28a03593c6537?cid=c5e5d6eec0ee45518a572285d4a12260'},
   'Jalo': {'id': '2sA7uCJUKTl6xsxLg0xQY2',
    'preview': 'https://p.scdn.co/mp3-preview/398c2cb27507b42686345845197cca75836fb88b?cid=c5e5d6eec0ee45518a572285d4a12260'},
   'Película': {'id': '4TCppHFDaH00Sc4Vf9QnDj',
    'preview': 'https://p.scdn.co/mp3-preview/4e4d0164901d43993c3e60345a3d22ea557b7391?cid=c5e5d6eec0ee45518a572285d4a12260'},
   'Yin Yang': {'id': '3bYttloZdsQGAj10cS5HEZ',
    'preview': 'https://p.scdn.co/mp3-preview/6b9066922f482df5ac5bbdc651ffb4c80c1878fd?cid=c5e5d6eec0ee45518a572285d4a12260'},
   'Tú': {'id': '536jZQnDc8UczLFWyRqEzt',
    'preview': 'https://p.scd

In [30]:
preview_url = albums_list['Yin']['tracks']['Conmigo Siempre']['preview']
preview = requests.get(preview_url)
print(preview.status_code)
import IPython.display as ipd
ipd.Audio(preview.content)

200


In [31]:
import csv
with open('test.csv', 'w') as f:
    for key in albums_list.keys():
        f.write("%s,%s\n"%(key,albums_list[key]))